In [15]:
import pandas as pd
from datetime import datetime

# Load the datasets
dataset = pd.read_csv('Datasetv2.csv')
property_data = pd.read_csv('Property data  raw 2.csv')

# Define the function to convert date to year-quarter format
def get_year_quarter(date_str):
    date_formats = ['%d-%m-%y', '%d/%m/%y', '%Y-%m-%d', '%m/%d/%y', '%d.%m.%Y', '%d-%m-%Y', '%d/%m/%Y']
    for fmt in date_formats:
        try:
            date = datetime.strptime(date_str, fmt)
            year = date.year
            quarter = (date.month - 1) // 3 + 1
            return date.strftime('%d-%m-%y'), f"{year}Q{quarter}"
        except ValueError:
            continue
    raise ValueError(f"Date format not recognized: {date_str}")

# Extract date and quarter info from the dataset
dataset[['date', 'YearQuarter']] = dataset['date'].apply(lambda x: pd.Series(get_year_quarter(x)))

# Melt the property data to have a long format
property_data_melted = property_data.melt(id_vars=[property_data.columns[0]], var_name='YearQuarter', value_name='Value')

# Pivot the melted property data to have a wide format
property_data_pivoted = property_data_melted.pivot(index='YearQuarter', columns=property_data.columns[0], values='Value').reset_index()

# Merge the datasets on the YearQuarter column
merged_dataset = pd.merge(dataset, property_data_pivoted, on='YearQuarter', how='left')

# Add quarter info in a new column
merged_dataset['Quarter'] = merged_dataset['YearQuarter'].apply(lambda x: x.split('Q')[1])

# Convert saleable_area(ft^2) and unit_rate to numeric types
merged_dataset['saleable_area(ft^2)'] = pd.to_numeric(merged_dataset['saleable_area(ft^2)'].str.replace(',', ''), errors='coerce')
merged_dataset['unit_rate'] = pd.to_numeric(merged_dataset['unit_rate'], errors='coerce')

# Create a new column for the revised price
merged_dataset['Revised Price'] = merged_dataset['saleable_area(ft^2)'] * merged_dataset['unit_rate']

# Save the merged dataset to a new CSV file
merged_dataset.to_csv('Merged_Datasetv2.csv', index=False)